In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import os
import torch 
import pytorch_lightning as pl

from pytorchvideo.data import LabeledVideoDataset
from pytorchvideo.data.clip_sampling import UniformClipSampler
from pytorchvideo.transforms import ApplyTransformToKey, UniformTemporalSubsample
from torchvision.transforms import Compose, Resize

from utils.config import TRAIN_LABELS_FILE, TEST_FOLDER, TRAIN_FOLDER

In [5]:
fname_label_pairs = []

with open(TRAIN_LABELS_FILE, 'r') as f:
    for line in f:
        try:
            fname, label = line.strip().split(',')
            fname = os.path.join(TRAIN_FOLDER, fname)
            # label = {"label":int(label)}
            label = {"label": torch.tensor([int(label)], dtype=torch.int32)}
            fname_label_pairs.append(tuple((fname, label)))
        except:
            pass

In [6]:
transform = Compose([
    ApplyTransformToKey(
        key="video",
        transform=Compose([
            UniformTemporalSubsample(16),
            Resize(128)
        ])
    )
])

In [7]:
train_dataset = LabeledVideoDataset(
    fname_label_pairs,
    UniformClipSampler(5.),
    transform=transform,
    decode_audio=False,
)

In [8]:
from torch.utils.data import  DataLoader

train_loader = DataLoader(train_dataset, batch_size=10)

In [9]:
loader = iter(train_loader)

In [10]:
batch = next(loader)

In [12]:
batch['label']

tensor([[0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0]], dtype=torch.int32)

In [35]:
batch['video'].dtype

torch.float32

In [186]:
from sklearn.model_selection import train_test_split

In [189]:
length = len(fname_label_pairs)

In [196]:
train, val = train_test_split(fname_label_pairs, test_size=0.1, train_size=0.9)

# Training

In [1]:
from pytorch_lightning import Trainer

from models.model import FakeVideoDetector, create_resnet
from utils.datasets import FakeVideoDataModule

Global seed set to 42


In [2]:
classifier = FakeVideoDetector()
datamodule = FakeVideoDataModule(num_workers=1, batch_size=4)

In [3]:
trainer = Trainer(gpus=1, precision=16)

Using native 16bit precision.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [4]:
trainer.fit(classifier, datamodule)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type              | Params
--------------------------------------------
0 | model | Net               | 31.7 M
1 | loss  | BCEWithLogitsLoss | 0     
--------------------------------------------
31.7 M    Trainable params
0         Non-trainable params
31.7 M    Total params
126.621   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/nazar/anaconda3/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Global seed set to 42
/home/nazar/anaconda3/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: -1it [00:00, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 32.00 MiB (GPU 0; 6.00 GiB total capacity; 4.66 GiB already allocated; 0 bytes free; 4.75 GiB reserved in total by PyTorch)